In [4]:
!pip install llama-index-core
!pip install llama-index-readers-file
!pip install llama-index-tools-wolfram-alpha
!pip install 'crewai[tools]'


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import LlamaIndexTool
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
!wget "https://s23.q4cdn.com/407969754/files/doc_financials/2019/ar/Uber-Technologies-Inc-2019-Annual-Report.pdf" -O uber_10k.pdf

--2025-02-08 11:16:19--  https://s23.q4cdn.com/407969754/files/doc_financials/2019/ar/Uber-Technologies-Inc-2019-Annual-Report.pdf
Resolving s23.q4cdn.com (s23.q4cdn.com)... 68.70.205.3, 68.70.205.2, 68.70.205.1, ...
Connecting to s23.q4cdn.com (s23.q4cdn.com)|68.70.205.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2829436 (2.7M) [application/pdf]
Saving to: ‘uber_10k.pdf’

uber_10k.pdf        100%[===================>]   2.70M  3.57MB/s    in 0.8s    

2025-02-08 11:16:20 (3.57 MB/s) - ‘uber_10k.pdf’ saved [2829436/2829436]



In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI

reader = SimpleDirectoryReader(input_files=["uber_10k.pdf"])
docs = reader.load_data()

In [3]:
docs[1].get_content()

'69 \nCountries\n10K+ \nCities\n$65B \nGross Bookings\n111M \nMAPCs\n7B \nTrips\nA global tech \nplatform at \nmassive scale\nServing multiple multi-trillion \ndollar markets with products \nleveraging our core technology \nand infrastructure\nWe believe deeply in our bold mission. Every minute \nof every day, consumers and Drivers on our platform \ncan tap a button and get a ride or tap a button and \nget work. We revolutionized personal mobility with \nridesharing, and we are leveraging our platform to \nredefine the massive meal delivery and logistics \nindustries. The foundation of our platform is our \nmassive network, leading technology, operational \nexcellence, and product expertise. Together, these \nelements power movement from point A to point B.'

In [6]:
llm = OpenAI(model="gpt-4o")
index = VectorStoreIndex.from_documents(docs)
query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)

In [7]:
# try out query engine tool
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Uber 2019 10K Query Tool",
    description="Use this tool to lookup the 2019 Uber 10K Annual Report",
)

In [12]:
query_tool.args_schema.schema()

{'description': 'Schema for query tool.',
 'properties': {'query': {'description': 'Search query for the query tool.',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'QueryToolSchema',
 'type': 'object'}

In [9]:
# Define your agents with roles and goals
researcher = Agent(
    role="Senior Financial Analyst",
    goal="Uncover insights about different tech companies",
    backstory="""You work at an asset management firm.
  Your goal is to understand tech stocks like Uber.""",
    verbose=True,
    allow_delegation=False,
    tools=[query_tool],
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=False,
)

# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of Uber's risk factors in 2019.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the headwinds that Uber faces.
  Your post should be informative yet accessible, catering to a casual audience.
  Make it sound cool, avoid complex words.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True,  # You can set it to 1 or 2 to different logging levels
)

In [10]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

# Agent: Senior Financial Analyst
## Task: Conduct a comprehensive analysis of Uber's risk factors in 2019.


# Agent: Senior Financial Analyst
## Thought: I need to find detailed information regarding Uber's risk factors from the 2019 10K Annual Report so that I can conduct a comprehensive analysis.
## Using tool: Uber 2019 10K Query Tool
## Tool Input: 
"{\"query\": \"risk factors\"}"
## Tool Output: 
The company faces several risk factors, including liabilities from data security breaches, such as those experienced by acquired companies like Careem. There are also risks related to the collection and processing of data, which could lead to investigations, litigation, and fines. Additionally, the company is exposed to claims from incidents involving drivers and consumers, which may not be fully covered by insurance. The expansion into new offerings and technologies, such as autonomous vehicles and dockless e-bikes, involves significant investment and inherent risks, including regulato